### Konstantina Andronikou

## This notebook is implementing Negation as a Robustness test 

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
#Removing the warnings 
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("John has pets.")
output

{'verbs': [{'verb': 'has',
   'description': '[ARG0: John] [V: has] [ARG1: pets] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['John', 'has', 'pets', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["John has pets"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'has',
     'description': '[ARG0: John] [V: has] [ARG1: pets]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1']}],
   'words': ['John', 'has', 'pets']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} has pets.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Wendy'}, {'first_name': 'John'}, {'first_name': 'Sophie'}, {'first_name': 'Frederick'}, {'first_name': 'Alan'}, {'first_name': 'Jay'}, {'first_name': 'Grace'}, {'first_name': 'Sandra'}, {'first_name': 'Kathleen'}, {'first_name': 'Carol'}, {'first_name': 'Melissa'}, {'first_name': 'Kathy'}, {'first_name': 'Anne'}, {'first_name': 'Francis'}, {'first_name': 'Stephanie'}, {'first_name': 'Simon'}, {'first_name': 'Patrick'}, {'first_name': 'Pamela'}, {'first_name': 'Rachel'}, {'first_name': 'Mark'}, {'first_name': 'Jimmy'}, {'first_name': 'Charlotte'}, {'first_name': 'Alan'}, {'first_name': 'Alfred'}, {'first_name': 'Lauren'}, {'first_name': 'Cynthia'}, {'first_name': 'Betty'}, {'first_name': 'Marie'}, {'first_name': 'Lauren'}, {'first_name': 'Nancy'}, {'first_name': 'Heather'}, {'first_name': 'Pamela'}, {'first_name': 'Greg'}, {'first_name': 'Ian'}, {'first_name': 'Janet'}, {'first_name': 'Charlotte'}, {'first_name': 'Scott'}, {'first_name': 'Fred'}, {'first_name': 'Ga

In [12]:
with open('dataset/Without_Negation.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Without_negation.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Wendy', 'has', 'pets', '.'] [ True]
['John', 'has', 'pets', '.'] [ True]
['Sophie', 'has', 'pets', '.'] [ True]
['Frederick', 'has', 'pets', '.'] [ True]
['Alan', 'has', 'pets', '.'] [ True]
['Jay', 'has', 'pets', '.'] [ True]
['Grace', 'has', 'pets', '.'] [ True]
['Sandra', 'has', 'pets', '.'] [ True]
['Kathleen', 'has', 'pets', '.'] [ True]
['Carol', 'has', 'pets', '.'] [ True]
['Melissa', 'has', 'pets', '.'] [ True]
['Kathy', 'has', 'pets', '.'] [ True]
['Anne', 'has', 'pets', '.'] [ True]
['Francis', 'has', 'pets', '.'] [ True]
['Stephanie', 'has', 'pets', '.'] [ True]
['Simon', 'has', 'pets', '.'] [ True]
['Patrick', 'has', 'pets', '.'] [ True]
['Pamela', 'has', 'pets', '.'] [ True]
['Rachel', 'has', 'pets', '.'] [ True]
['Mark', 'has', 'pets', '.'] [ True]
['Jimmy', 'has', 'pets', '.'] [ True]
['Charlotte', 'has', 'pets', '.'] [ True]
['Alan', 'has', 'pets', '.'] [ True]
['Alfred', 'has', 'pets', '.'] [ Tr

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_without_negation.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Adding Negation 

In [18]:
editor = Editor()

# create examples
t = editor.template('{first_name} has no pets.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Eric'}, {'first_name': 'Philip'}, {'first_name': 'Sam'}, {'first_name': 'Lucy'}, {'first_name': 'Robin'}, {'first_name': 'Daniel'}, {'first_name': 'Adam'}, {'first_name': 'Alfred'}, {'first_name': 'Don'}, {'first_name': 'Florence'}, {'first_name': 'Ken'}, {'first_name': 'Alexandra'}, {'first_name': 'Harold'}, {'first_name': 'Michael'}, {'first_name': 'Jane'}, {'first_name': 'Ron'}, {'first_name': 'Jimmy'}, {'first_name': 'Jane'}, {'first_name': 'Anthony'}, {'first_name': 'Mary'}, {'first_name': 'Victoria'}, {'first_name': 'Fiona'}, {'first_name': 'Sarah'}, {'first_name': 'Ralph'}, {'first_name': 'Tony'}, {'first_name': 'Jerry'}, {'first_name': 'Francis'}, {'first_name': 'Lauren'}, {'first_name': 'Jerry'}, {'first_name': 'Donna'}, {'first_name': 'Andrew'}, {'first_name': 'Philip'}, {'first_name': 'Howard'}, {'first_name': 'Ralph'}, {'first_name': 'James'}, {'first_name': 'Sue'}, {'first_name': 'Karen'}, {'first_name': 'Jeff'}, {'first_name': 'Benjamin'}, {'first_na

In [19]:
with open('dataset/With_Negation.txt', 'w') as f:
    print(t.data, file = f)

In [20]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/With_Negation.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    4 (0.4%)

Example fails:
[ARGM-DIS: Andrea] [V: has] [ARG1: no pets] .
----
[ARGM-DIS: Andrea] [V: has] [ARG1: no pets] .
----
[ARGM-DIS: Andrea] [V: has] [ARG1: no pets] .
----
['Eric', 'has', 'no', 'pets', '.'] [ True]
['Philip', 'has', 'no', 'pets', '.'] [ True]
['Sam', 'has', 'no', 'pets', '.'] [ True]
['Lucy', 'has', 'no', 'pets', '.'] [ True]
['Robin', 'has', 'no', 'pets', '.'] [ True]
['Daniel', 'has', 'no', 'pets', '.'] [ True]
['Adam', 'has', 'no', 'pets', '.'] [ True]
['Alfred', 'has', 'no', 'pets', '.'] [ True]
['Don', 'has', 'no', 'pets', '.'] [ True]
['Florence', 'has', 'no', 'pets', '.'] [ True]
['Ken', 'has', 'no', 'pets', '.'] [ True]
['Alexandra', 'has', 'no', 'pets', '.'] [ True]
['Harold', 'has', 'no', 'pets', '.'] [ True]
['Michael', 'has', 'no', 'pets', '.'] [ True]
['Jane', 'has', 'no', 'pets', '.'] [ True]
['Ron', 'has', 'no', 'pets', '.'] [ True]
['Jimmy', 'has', 'no', 'pets', '.'] [ True]
['Jane', 

In [21]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_with_negation.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 